# INTRODUCTION

<h2>Énoncé du problème</h2>

la deuxième plus grande chaîne de pharmacies allemandes, a mis au défi ses concurrents de prédire 6 semaines de ventes quotidiennes pour 1 115 magasins situés à travers l'Allemagne. 
Selon les informations fournies, les ventes sont influencées par de nombreux facteurs, notamment les promotions, la concurrence, les vacances scolaires et nationales, la saisonnalité et la localité.
Dans cet article, nous commençons par récupérer les données de Kaggle.
Ensuite, nous explorons le contenu des données. Bien qu'il s'agisse de données claires principalement structurées, nous aurons quelques défis en cours de route.
En utilisant d'excellents outils de gestion de données Python (Pandas, Numpy, etc.), nous les surmontons rapidement.
La publication se termine après la prédiction des ventes pour les données de test. Ainsi, nous avons une solution complète à un problème marketing réel (avec des données en bon état).

<h2> Objectif </h2>

   Notre objectif principal est de prédire les ventes quotidiennes du magasin Rossmann, les ventes sont influencées par de nombreux facteurs, notamment les promotions, la concurrence, les vacances scolaires et nationales, la saisonnalité et la localité.
 
Nous voulons minimiser autant que possible la métrique donnée.

<img src= "https://miro.medium.com/max/970/1*S4a5Chz3ldbu8xX6YOM8lA.png" alt ="Error_Matric" style='width: 100%;'>


<h2> Importer des bibliothèques </h2>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import (ensemble, preprocessing, tree,)
from sklearn.metrics import (auc, confusion_matrix, roc_auc_score, roc_curve,)
from sklearn.model_selection import (train_test_split, StratifiedKFold,)
from yellowbrick.classifier import (ConfusionMatrix, ROCAUC,)
from yellowbrick.model_selection import (LearningCurve,)

import pandas_profiling

import statistics
from statistics import mode

<h2> Importer les données </h2>


In [ ]:
train = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv", low_memory=False)
test = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv", low_memory=False)
store = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv", low_memory=False)

In [ ]:
test.head() #afficher les 5 premières lignes de test dataset

In [ ]:
train.head() #afficher les 5 premières lignes de train dataset

In [ ]:
store.head() #afficher les 5 premières lignes de store dataset

<h2>  EDA ( Exploratory Data Analysis  ) </h2>

<h3>1- Eplorer les données </h3>

In [ ]:
print(f'Pour dataset train on a (lines,attributes) = {train.shape}. \n')
print('*'*70 + '\n')

train.info()

In [ ]:
print(f'Pour dataset stores on a (lines,attributes) = {store.shape}. \n')
print('*'*70 + '\n')

store.info()

In [ ]:
store['StoreType'].unique()

In [ ]:
store['Assortment'].unique()

In [ ]:
store['PromoInterval'].unique()


* Sales : the turnover for a given day
* Customers : the number of customers on a given day.
* Open : an indicator for whether the store was open: 0 = closed, 1 = open.
* StateHoliday : indicates a state holiday, a = public holiday, b = Easter holiday, c = Christmas, 0 = None.
* StoreType : differentiates between 4 different store models: a, b, c, d, a = basic, b = extra, c = extended.
* CompetitionDistance : distance in meters to the nearest competitor store.
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened.
* Promo : indicates whether a store is running a promo on that day.
* Promo2 : Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating.

<h3>2-  Cleaning the DATA </h3>

In [ ]:
#pour un magasin qui n'est pas ouvert la mean est 0.0
train.loc[train['Open'] == 0, 'Sales'].mean()

* <h5> 2-1-1 NULL AND NAN </h5>

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
store.isnull().sum()

# TEST

In [ ]:
test.loc[np.isnan(test['Open']), 'Open'] = mode(test['Open'])

In [ ]:
test.isnull().sum()

# STORE

In [ ]:
store.loc[np.isnan(store['CompetitionDistance']), 'CompetitionDistance'] = store['CompetitionDistance'].median()
#store.loc[np.isnan(store['CompetitionOpenSinceYear']), 'CompetitionOpenSinceYear'] = mode(store['CompetitionOpenSinceYear'])
#store.loc[np.isnan(store['Promo2SinceWeek']), 'Promo2SinceWeek'] = mode(store['Promo2SinceWeek'])

In [ ]:
store.fillna(0,inplace=True)

In [ ]:
store.isnull().sum()

* <h5> 2-1-1 convert data to integers </h5>

# Merge data

In [ ]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

test.reset_index(drop=True, inplace=True)
test.set_index('Id', inplace=True)
test.reset_index()

In [ ]:
def cleaning_data(data):
    
    data['Year'] = pd.DatetimeIndex(data['Date']).year
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    data['Day'] = pd.DatetimeIndex(data['Date']).day    
    data['QuadYear'] = pd.DatetimeIndex(data['Date']).quarter
    data['DayOfYear'] = pd.DatetimeIndex(data['Date']).day_of_year

    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    mappings2 = {'0':0, 'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2, 'Mar,Jun,Sept,Dec':3}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    data.PromoInterval.replace(mappings2, inplace=True)
    data['StoreType'] = data['StoreType'].astype(int)
    data['Assortment'] = data['Assortment'].astype(int)
    data['StateHoliday'] = data['StateHoliday'].astype(int)
    data['PromoInterval'] = data['PromoInterval'].astype(int)

In [ ]:
cleaning_data(train)

In [ ]:
cleaning_data(test)

In [ ]:
def cleaning_store(data):
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    mappings2 = {'0':0, 'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2, 'Mar,Jun,Sept,Dec':3}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.PromoInterval.replace(mappings2, inplace=True)
    data['StoreType'] = data['StoreType'].astype(int)
    data['Assortment'] = data['Assortment'].astype(int)
    data['PromoInterval'] = data['PromoInterval'].astype(int)

In [ ]:
#cleaning_store(store)

<h3> 3- Analyse Relationship </h3>

In [ ]:
corrocaltion = pd.DataFrame(train.loc[:, ['DayOfWeek', 'Sales', 'Month', 'Year', 'Customers', 'Promo', 'StateHoliday', 'SchoolHoliday']].corr())
print(corrocaltion)

In [ ]:
sns.heatmap(data=corrocaltion,vmax=.7,cbar=True,annot=True)
plt.show()

<h3> Week days effect</h3>

In [ ]:
fig, (axis1) = plt.subplots(1,1,figsize=(15,4))
sns.countplot(x='Open',hue='DayOfWeek', data=train,palette="husl", ax=axis1)

Cela montre clairement que la plupart des magasins restent fermés le dimanche. Certains magasins étaient également fermés en semaine, cela peut être dû aux jours fériés, car les magasins sont généralement fermés pendant les jours fériés et ouverts pendant les vacances scolaires. 


In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='DayOfWeek', y='Sales', data=train, order=[1,2,3,4,5,6,7], ax=axis1)
sns.barplot(x='DayOfWeek', y='Customers', data=train, order=[1,2,3,4,5,6,7], ax=axis2)

This clearly shows most sales are done with the first days, but very less on the last day due to the closed shops on sunday

<h3>  Promo VS sales </h3> 

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(20,4))

axis1.title.set_text('Promo vs.Sales')
axis2.title.set_text('Promo vs.Customers')
sns.barplot(x='Promo', y='Sales', data=train, ax=axis1)
sns.barplot(x='Promo', y='Customers', data=train, ax=axis2)

Ici, nous observons que les ventes et les clients augmentent de manière significative pendant les promotions. Cela montre que la promotion a un effet positif pour un magasin.

<h3>  Sales Vs Holidays </h3> 

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='StateHoliday', y='Sales', data= train, ax=axis1)
sns.barplot(x='StateHoliday', y='Customers', data= train, ax=axis2)

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))

sns.barplot(x='StoreType', y='Sales', data= train, ax=axis1)
sns.barplot(x='StoreType', y='Customers', data= train, ax=axis2)

<h3> Sales per Year </h3>

In [ ]:
train.boxplot(column='Sales', by='Year',)
plt.show()

<h3>  Sales per months </h3> 

In [ ]:
train.boxplot(column='Sales', by='Month',)
plt.show()

<h2> Conclusion </h2>

Conclusions de l'EDA : Le type de magasin le plus vendu et le plus fréquenté est A.

Les ventes sont fortement corrélées au nombre de clients.

Pour tous les magasins, la promotion entraîne une augmentation des ventes et des clients.

Les magasins ouverts pendant les vacances scolaires ont plus de ventes que les jours normaux.

Chaque fois qu'un magasin participe à une promotion, cela se fait pendant 5 jours et deux fois par mois pour chaque magasin.

Plus de magasins sont ouverts pendant les vacances scolaires que les jours fériés.

Les ventes augmentent pendant la semaine de Noël, cela peut être dû au fait que les gens achètent plus de produits de beauté lors d'une fête de Noël.

L'absence de valeurs dans les entités CompetitionOpenSinceYear/Month n'indique pas l'absence de concurrence car les valeurs CompetitionDistance ne sont pas nulles là où les deux autres valeurs sont nulles.

Après avoir analysé les ventes à l'aide de la décomposition de Fourier, j'ai découvert qu'il existe une composante de saisonnalité dans les données de ventes.

# MODELING

<h3> prétraitement des données </h3>

In [ ]:
test.head()

In [ ]:
train = train.loc[train.Open == 1]
train = train.loc[train.Sales > 0]
train.loc[train.Sales > 0].reset_index(drop=True)

In [ ]:
train = train.drop(columns="Date")
test = test.drop(columns="Date")

In [ ]:
y = train.Sales

In [ ]:
train = train.drop(columns = 'Sales')
train = train.drop(columns = 'Customers')
train.reset_index(drop=True, inplace=True)

In [ ]:
X = train

<h3> Spliting DATA </h3>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

<h3> Dummy Model </h3>

In [ ]:
y.head()

In [ ]:
from sklearn.dummy import DummyClassifier

bm= DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) # accuracy

<h3> Linear Regression Model </h3>

In [ ]:
from sklearn.linear_model import LinearRegression
 
linear = LinearRegression()
linear.fit(X_train, y_train)
linear.score(X_test, y_test)

<h3> Random Forest Regressor Model </h3>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1, random_state=42, n_estimators=200)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
rf.predict(test)

In [ ]:
test['Sales'] = rf.predict(test)

In [ ]:
submission = test['Sales']
submission = submission.reset_index()
submission.head()

In [ ]:
submission.to_csv("./submission_open.csv", sep=',', index=False)